# Analyse foncière Inter-Annuel

Importation des librairies et des données ainsi que l'import des dapartements et des régions


In [20]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import datetime
import json
import folium
import json
import pandas as pd
import numpy as np
from branca.colormap import linear
from datetime import timedelta
from urllib.request import urlopen
import numpy as np
import pandas as pd
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
import plotly.graph_objects as go
import plotly.express as px
import plotly.subplots as sp
import warnings
warnings.filterwarnings('ignore')

DEPARTMENTS = {
    '1': 'Ain', 
    '2': 'Aisne', 
    '3': 'Allier', 
    '4': 'Alpes-de-Haute-Provence', 
    '5': 'Hautes-Alpes',
    '6': 'Alpes-Maritimes', 
    '7': 'Ardèche', 
    '8': 'Ardennes', 
    '9': 'Ariège', 
    '10': 'Aube', 
    '11': 'Aude',
    '12': 'Aveyron', 
    '13': 'Bouches-du-Rhône', 
    '14': 'Calvados', 
    '15': 'Cantal', 
    '16': 'Charente',
    '17': 'Charente-Maritime', 
    '18': 'Cher', 
    '19': 'Corrèze', 
    '2A': 'Corse-du-Sud', 
    '2B': 'Haute-Corse',
    '21': 'Côte-d\'Or', 
    '22': 'Côtes-d\'Armor', 
    '23': 'Creuse', 
    '24': 'Dordogne', 
    '25': 'Doubs', 
    '26': 'Drôme',
    '27': 'Eure', 
    '28': 'Eure-et-Loir', 
    '29': 'Finistère', 
    '30': 'Gard', 
    '31': 'Haute-Garonne', 
    '32': 'Gers',
    '33': 'Gironde', 
    '34': 'Hérault', 
    '35': 'Ille-et-Vilaine', 
    '36': 'Indre', 
    '37': 'Indre-et-Loire',
    '38': 'Isère', 
    '39': 'Jura', 
    '40': 'Landes', 
    '41': 'Loir-et-Cher', 
    '42': 'Loire', 
    '43': 'Haute-Loire',
    '44': 'Loire-Atlantique', 
    '45': 'Loiret', 
    '46': 'Lot', 
    '47': 'Lot-et-Garonne', 
    '48': 'Lozère',
    '49': 'Maine-et-Loire', 
    '50': 'Manche', 
    '51': 'Marne', 
    '52': 'Haute-Marne', 
    '53': 'Mayenne',
    '54': 'Meurthe-et-Moselle', 
    '55': 'Meuse', 
    '56': 'Morbihan', 
    '57': 'Moselle', 
    '58': 'Nièvre', 
    '59': 'Nord',
    '60': 'Oise', 
    '61': 'Orne', 
    '62': 'Pas-de-Calais', 
    '63': 'Puy-de-Dôme', 
    '64': 'Pyrénées-Atlantiques',
    '65': 'Hautes-Pyrénées', 
    '66': 'Pyrénées-Orientales', 
    '67': 'Bas-Rhin', 
    '68': 'Haut-Rhin', 
    '69': 'Rhône',
    '70': 'Haute-Saône', 
    '71': 'Saône-et-Loire', 
    '72': 'Sarthe', 
    '73': 'Savoie', 
    '74': 'Haute-Savoie',
    '75': 'Paris', 
    '76': 'Seine-Maritime', 
    '77': 'Seine-et-Marne', 
    '78': 'Yvelines', 
    '79': 'Deux-Sèvres',
    '80': 'Somme', 
    '81': 'Tarn', 
    '82': 'Tarn-et-Garonne', 
    '83': 'Var', 
    '84': 'Vaucluse', 
    '85': 'Vendée',
    '86': 'Vienne', 
    '87': 'Haute-Vienne', 
    '88': 'Vosges', 
    '89': 'Yonne', 
    '90': 'Territoire de Belfort',
    '91': 'Essonne', 
    '92': 'Hauts-de-Seine', 
    '93': 'Seine-Saint-Denis', 
    '94': 'Val-de-Marne', 
    '95': 'Val-d\'Oise',
    '971': 'Guadeloupe', 
    '972': 'Martinique', 
    '973': 'Guyane', 
    '974': 'La Réunion', 
    '976': 'Mayotte',
}

REGIONS = {
    'Auvergne-Rhône-Alpes': ['01', '03', '07', '15', '26', '38', '42', '43', '63', '69', '73', '74'],
    'Bourgogne-Franche-Comté': ['21', '25', '39', '58', '70', '71', '89', '90'],
    'Bretagne': ['22', '29', '35', '56'],
    'Centre-Val de Loire': ['18', '28', '36', '37', '41', '45'],
    'Corse': ['2A', '2B'],
    'Grand Est': ['08', '10', '51', '52', '54', '55', '57', '67', '68', '88'],
    'Hauts-de-France': ['02', '59', '60', '62', '80'],
    'Île-de-France': ['75', '77', '78', '91', '92', '93', '94', '95'],
    'Normandie': ['14', '27', '50', '61', '76'],
    'Nouvelle-Aquitaine': ['16', '17', '19', '23', '24', '33', '40', '47', '64', '79', '86', '87'],
    'Occitanie': ['09', '11', '12', '30', '31', '32', '34', '46', '48', '65', '66', '81', '82'],
    'Pays de la Loire': ['44', '49', '53', '72', '85'],
    'Provence-Alpes-Côte d\'Azur': ['04', '05', '06', '13', '83', '84'],
    'Guadeloupe': ['971'],
    'Martinique': ['972'],
    'Guyane': ['973'],
    'La Réunion': ['974'],
    'Mayotte': ['976']
}

dth = '#ff2e63' 
rec = '#21bf73'
act = '#fe9801'

data = pd.read_csv(r"../data/annee_traitee/2018.csv",low_memory=False,sep=",",header=0,index_col=0)
data = data.append(pd.read_csv(r"../data/annee_traitee/2019.csv",low_memory=False,sep=",",header=0,index_col=0))
data = data.append(pd.read_csv(r"../data/annee_traitee/2020.csv",low_memory=False,sep=",",header=0,index_col=0))
data = data.append(pd.read_csv(r"../data/annee_traitee/2021.csv",low_memory=False,sep=",",header=0,index_col=0))
data = data.append(pd.read_csv(r"../data/annee_traitee/2022.csv",low_memory=False,sep=",",header=0,index_col=0))

data["Date mutation"] = pd.to_datetime(data["Date mutation"], format="%Y-%m-%d")
data = data.reset_index(drop=True)

In [21]:
display(data)

,Date mutation,Valeur fonciere,No voie,Type de voie,Code voie,Voie,Code postal,Code departement,Code commune,Section,No plan,1er lot,Surface Carrez du 1er lot,Nombre de lots,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Surface terrain
0,2018-01-03,109000.0,13.0,RUE,1660,GEN LOGEROT,1000.0,1,53,AN,73,1,NaN,2,2.0,Appartement,73.0,4.0,NaN,NaN
1,2018-01-03,109000.0,13.0,RUE,1660,GEN LOGEROT,1000.0,1,53,AN,73,13,NaN,1,3.0,Dépendance,0.0,0.0,NaN,NaN
2,2018-01-04,239300.0,4.0,RUE,0025,DE LA BARMETTE,1250.0,1,95,AH,186,NaN,NaN,0,1.0,Maison,163.0,4.0,S,949.0
3,2018-01-04,239300.0,4.0,RUE,0025,DE LA BARMETTE,1250.0,1,95,AH,186,NaN,NaN,0,1.0,Maison,163.0,4.0,AG,420.0
4,2018-01-04,239300.0,4.0,RUE,0025,DE LA BARMETTE,1250.0,1,95,AH,186,NaN,NaN,0,1.0,Maison,51.0,2.0,AG,420.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10758898,2022-12-30,3500000.0,19.0,RUE,4879,JEAN DOLENT,75014.0,75,114,AS,41,3,NaN,1,4.0,Local industriel. commercial ou assimilé,327.0,0.0,NaN,NaN
10758899,2022-12-19,525000.0,3.0,RUE,3842,FREDERIC MAGISSON,75015.0,75,115,EJ,51,49,46.65,2,3.0,Dépendance,0.0,0.0,NaN,NaN
10758900,2022-12-19,525000.0,3.0,RUE,3842,FREDERIC MAGISSON,75015.0,75,115,EJ,51,49,46.65,2,2.0,Appartement,40.0,3.0,NaN,NaN
10758901,2022-12-30,15500.0,69.0,RUE,0679,BARRAULT,75013.0,75,113,DM,28,244,NaN,1,3.0,Dépendance,0.0,0.0,NaN,NaN


In [22]:
temp = data[(data["Type local"] != "Dépendance")& (data["Type local"] != "Local industriel. commercial ou assimilé")].reset_index(drop = True)
temp["Month mutation"] = temp["Date mutation"].dt.month

temp['Prix mètre carré Paris 2018'] = temp[(temp["Code departement"] == '75')&(temp["Date mutation"].dt.year == 2018)]["Valeur fonciere"]/temp[(temp["Code departement"] == '75')&(temp["Date mutation"].dt.year == 2018)]["Surface reelle bati"]
temp['Prix mètre carré Paris 2019'] = temp[(temp["Code departement"] == '75')&(temp["Date mutation"].dt.year == 2019)]["Valeur fonciere"]/temp[(temp["Code departement"] == '75')&(temp["Date mutation"].dt.year == 2019)]["Surface reelle bati"]
temp['Prix mètre carré Paris 2022'] = temp[(temp["Code departement"] == '75')&(temp["Date mutation"].dt.year == 2022)]["Valeur fonciere"]/temp[(temp["Code departement"] == '75')&(temp["Date mutation"].dt.year == 2022)]["Surface reelle bati"]

temp =temp.replace(np.inf, np.nan)
display(temp['Prix mètre carré Paris 2018'].mean())
display(temp)

# temp['No. of Recovered to 1 Death Case'] = round(temp['Recovered']/temp['Deaths'], 3)
temp = temp.groupby('Month mutation')["Prix mètre carré Paris 2018","Prix mètre carré Paris 2019","Prix mètre carré Paris 2022"].mean().reset_index()

temp = temp.melt(id_vars='Month mutation', value_vars=['Prix mètre carré Paris 2018','Prix mètre carré Paris 2019','Prix mètre carré Paris 2022'], 
                 var_name='Departements', value_name='Value')

fig = px.line(temp, x="Month mutation", y="Value", color='Departements', log_y=True, 
              title='Evolution du prix du mètre carré à paris depuis 2018', color_discrete_sequence=[dth, rec,act])
fig.show()



39946.7181677061

,Date mutation,Valeur fonciere,No voie,Type de voie,Code voie,Voie,Code postal,Code departement,Code commune,Section,...,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Surface terrain,Month mutation,Prix mètre carré Paris 2018,Prix mètre carré Paris 2019,Prix mètre carré Paris 2022
0,2018-01-03,109000.0,13.0,RUE,1660,GEN LOGEROT,1000.0,1,53,AN,...,2.0,Appartement,73.0,4.0,NaN,NaN,1,NaN,NaN,NaN
1,2018-01-04,239300.0,4.0,RUE,0025,DE LA BARMETTE,1250.0,1,95,AH,...,1.0,Maison,163.0,4.0,S,949.0,1,NaN,NaN,NaN
2,2018-01-04,239300.0,4.0,RUE,0025,DE LA BARMETTE,1250.0,1,95,AH,...,1.0,Maison,163.0,4.0,AG,420.0,1,NaN,NaN,NaN
3,2018-01-04,239300.0,4.0,RUE,0025,DE LA BARMETTE,1250.0,1,95,AH,...,1.0,Maison,51.0,2.0,AG,420.0,1,NaN,NaN,NaN
4,2018-01-04,239300.0,4.0,RUE,0025,DE LA BARMETTE,1250.0,1,95,AH,...,1.0,Maison,51.0,2.0,S,949.0,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6507941,2022-12-29,1650000.0,78.0,AV,7149,PAUL DOUMER,75016.0,75,116,DM,...,2.0,Appartement,150.0,6.0,NaN,NaN,12,NaN,NaN,11000.000000
6507942,2022-11-03,291000.0,128.0,RUE,0012,DE L ABBE GROULT,75015.0,75,115,AE,...,2.0,Appartement,35.0,2.0,NaN,NaN,11,NaN,NaN,8314.285714
6507943,2022-12-23,934000.0,5.0,VLA,3488,FAISANDERIE,75016.0,75,116,EE,...,2.0,Appartement,58.0,2.0,NaN,NaN,12,NaN,NaN,16103.448276
6507944,2022-12-23,70000.0,67.0,RUE,1255,BRANCION,75015.0,75,115,AJ,...,2.0,Appartement,15.0,1.0,NaN,NaN,12,NaN,NaN,4666.666667
